In [10]:
import os
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

scopes = ["https://www.googleapis.com/auth/spreadsheets"]
spreadsheet_id = "YOUR_SPREADSHEET_ID"
credential_path = r"path_to_OAUTH_JSON_FILE"

credentials = None
if os.path.exists("token.json"):
    credentials = Credentials.from_authorized_user_file("token.json", scopes)
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(credential_path, scopes)
        credentials = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
        token.write(credentials.to_json())


service = build("sheets", "v4", credentials=credentials)
result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range="Sheet1!A:J").execute()
values = result.get("values", [])

# Create a DataFrame from the values
df = pd.DataFrame(values)
df.columns = df.iloc[0]
df = df[1:]
